In [77]:

import pandas as pd
import numpy as np
import main as fca
from sklearn.model_selection import train_test_split
from sklearn import preprocessing, metrics
from importlib import reload
import warnings
import pandas as pd
from pandas.core.common import SettingWithCopyWarning
import lazy_pipeline as lpipe
import folium
from folium.plugins import HeatMap
import plotly.express as px
import seaborn as sns
import missingno as msno
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score,recall_score,precision_score,confusion_matrix,plot_roc_curve,classification_report
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
from xgboost import XGBClassifier
from catboost import CatBoostClassifier
from sklearn.ensemble import ExtraTreesClassifier
from lightgbm import LGBMClassifier
from sklearn.ensemble import VotingClassifier

import warnings
warnings.filterwarnings('ignore')

plt.style.use('fivethirtyeight')
%matplotlib inline
pd.set_option('display.max_columns', 32)

warnings.simplefilter(action="ignore", category=SettingWithCopyWarning)

In [78]:
#data https://www.kaggle.com/datasets/ketangangal/heart-disease-dataset-uci/code
df = pd.read_csv('heart_disease.csv')
df.head()

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,condition
0,69,1,0,160,234,1,2,131,0,0.1,1,1,0,0
1,69,0,0,140,239,0,0,151,0,1.8,0,2,0,0
2,66,0,0,150,226,0,0,114,0,2.6,2,0,0,0
3,65,1,0,138,282,1,2,174,0,1.4,1,1,0,1
4,64,1,0,110,211,0,2,144,1,1.8,1,0,0,0


In [79]:
df.isnull().any()

age          False
sex          False
cp           False
trestbps     False
chol         False
fbs          False
restecg      False
thalach      False
exang        False
oldpeak      False
slope        False
ca           False
thal         False
condition    False
dtype: bool

In [81]:
df.dtypes

age            int64
sex            int64
cp             int64
trestbps       int64
chol           int64
fbs            int64
restecg        int64
thalach        int64
exang          int64
oldpeak      float64
slope          int64
ca             int64
thal           int64
condition      int64
dtype: object

In [82]:
df = df.dropna()
df.head()
cols = list(df.columns)
cols.remove('condition')
X = df.loc[:,cols]
y = df.condition
X.shape,y.shape

((297, 13), (297,))

In [83]:
X = X.sample(frac=1, random_state=42)  # Shuffle X
y = y[X.index]

In [84]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.6, random_state = 1)

In [85]:
import main as fca

In [86]:
fca.fitClassifier(X_train, y_train)

In [87]:
y_pred = fca.predict(X_test)

In [88]:
results = {'accuracy': [], 'precision': [], 'recall': []}
results['accuracy'].append(metrics.accuracy_score(y_test, y_pred))
results['precision'].append(metrics.precision_score(y_test, y_pred))
results['recall'].append(metrics.recall_score(y_test, y_pred))

print('Accuracy:', pd.DataFrame(results)['accuracy'])

Accuracy: 0    0.798883
Name: accuracy, dtype: float64


In [89]:

classifiers = [
    DecisionTreeClassifier(),
    RandomForestClassifier(),
    AdaBoostClassifier(),
    GradientBoostingClassifier(),
    ]

log_cols=["Classifier", "Accuracy", "Precision", "Recall"]
log = pd.DataFrame(columns=log_cols)


In [90]:
for clf in classifiers:
    clf.fit(X_train, y_train)
    name = clf.__class__.__name__
    
    train_predictions = clf.predict(X_test)
    acc = accuracy_score(y_test, train_predictions)
    
    train_predictions = clf.predict(X_test)
    prec = precision_score(y_test, train_predictions)
    
    
    train_predictions = clf.predict(X_test)
    rec = recall_score(y_test, train_predictions)
      
    
    train_predictions = clf.predict_proba(X_test)
    
    
    log_entry = pd.DataFrame([[name, acc, prec, rec]], columns=log_cols)
    log = log.append(log_entry)
    
print("="*30)

In [91]:
print(log)

                   Classifier  Accuracy  Precision    Recall
0      DecisionTreeClassifier  0.743017   0.743590  0.690476
0      RandomForestClassifier  0.798883   0.833333  0.714286
0          AdaBoostClassifier  0.787709   0.802632  0.726190
0  GradientBoostingClassifier  0.765363   0.776316  0.702381


In [92]:
f_acc,f_pre,f_rec = results['accuracy'][0],results['precision'][0],results['recall'][0]
fcaa_row = {'Classifier': 'FCA', 'Accuracy': f_acc, 'Precision': f_pre,'Recall':f_rec}
log = log.append(fcaa_row, ignore_index = True)
  
display(log)

,Classifier,Accuracy,Precision,Recall
0,DecisionTreeClassifier,0.743017,0.743590,0.690476
1,RandomForestClassifier,0.798883,0.833333,0.714286
2,AdaBoostClassifier,0.787709,0.802632,0.726190
3,GradientBoostingClassifier,0.765363,0.776316,0.702381
4,FCA,0.798883,0.800000,0.761905


In [93]:
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px

log.sort_values(by = 'Accuracy', ascending = False)
fig = px.bar(data_frame = log, x = 'Accuracy', y = 'Classifier', color = 'Accuracy', template = 'ggplot2', title = 'Models Comparison')
fig.show()